# NLP with Disaster Tweets (on Kaggle)

In another notebook, I have compared different models in predicting disaster tweets. I found transformers BERT model give me the best accuracy. Here, I'm trying to use bagging to further improve the performance.

### Import libraries and import data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.4 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model
from transformers import AutoTokenizer, TFAutoModel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
path = "/content/drive/MyDrive/Colab Notebooks/DisasterTweets/data/"
train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")

In [5]:
import re

In [6]:
remove_url = lambda x:re.sub(r'https?://\S+|www\.\S+', '', x)
train_df['text'] = train_df['text'].map(remove_url)
test_df['text'] = test_df['text'].map(remove_url)

In [7]:
train_sentences = train_df['text'].to_numpy()
test_sentences = test_df['text'].to_numpy()
y = train_df['target'].to_numpy()

In [8]:
# number of training samples
n = len(train_sentences)
print("number of trainning samples: {}".format(n))

number of trainning samples: 7613


### Functions for sampling, tokenizer, and building models

In [9]:
# create a function to randomly choose samples
def resample(sample, target, sample_size=0.7):
  if not isinstance(sample, np.ndarray):
    sample = np.array(sample)
  if not isinstance(target, np.ndarray):
    target = np.array(target)
  n = len(sample)
  idx_selector = np.random.choice(n,int(sample_size*n), replace=False)
  not_selected = list(set(range(n)) - set(idx_selector))
  return sample[idx_selector], target[idx_selector], sample[not_selected], target[not_selected]

In [10]:
checkpoint = 'bert-base-uncased'
max_length=33
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_samples(samples):
  if isinstance(samples, np.ndarray):
    samples = samples.tolist()
  
  return tokenizer(
      text=samples,
      add_special_tokens=True,
      max_length=max_length,
      truncation=True,
      padding=True,
      return_tensors='tf',
      return_token_type_ids=False,
      return_attention_mask=True,
      verbose=True
    )

X_test = tokenize_samples(test_sentences)

In [11]:
# write a function to create and compile model
def create_compile_model(checkpoint=checkpoint):
  pretrained_model = TFAutoModel.from_pretrained(checkpoint)
  input_ids = layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
  input_mask = layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
  embeddings = pretrained_model(input_ids, attention_mask = input_mask)[1]
  x = layers.Dropout(0.3)(embeddings)
  x = layers.Dense(64, activation='relu')(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(32, activation='relu')(x)
  output = layers.Dense(1, activation='sigmoid')(x)
  model = Model(inputs=[input_ids, input_mask], outputs=output)
  model.compile(
      loss='binary_crossentropy',
      optimizer = Adam(learning_rate=5e-5),
      metrics=['accuracy']
  )
  return model

### Combine models to make prediction

In [12]:
models = []
y_preds = []
n_models = 15
for i in range(n_models):
  train_samples, y_train, val_samples, y_val = resample(train_sentences, y)
  X_train = tokenize_samples(train_samples)
  X_val = tokenize_samples(val_samples)
  model = create_compile_model()
  model.fit(
      [X_train['input_ids'], X_train['attention_mask']],
      y_train,
      epochs=10,
      validation_data = ([X_val['input_ids'], X_val['attention_mask']], y_val)
  )
  models.append(model)
  probs = model.predict([X_test['input_ids'], X_test['attention_mask']]).flatten()
  pred = (probs > 0.5).astype('int')
  y_preds.append(pred)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/10
167/167 [==============================] - 30s 81ms/step - loss: 0.5059 - accuracy: 0.7686 - val_loss: 0.4554 - val_accuracy: 0.8170
Epoch 2/10
167/167 [==============================] - 11s 64ms/step - loss: 0.3517 - accuracy: 0.8636 - val_loss: 0.4443 - val_accuracy: 0.8301
Epoch 3/10
167/167 [==============================] - 11s 64ms/step - loss: 0.2562 - accuracy: 0.9045 - val_loss: 0.4419 - val_accuracy: 0.8301
Epoch 4/10
167/167 [==============================] - 11s 64ms/step - loss: 0.1705 - accuracy: 0.9422 - val_loss: 0.6286 - val_accuracy: 0.8091
Epoch 5/10
167/167 [==============================] - 11s 64ms/step - loss: 0.1093 - accuracy: 0.9602 - val_loss: 0.7540 - val_accuracy: 0.8017
Epoch 6/10
167/167 [==============================] - 11s 64ms/step - loss: 0.0716 - accuracy: 0.9726 - val_loss: 0.8113 - val_accuracy: 0.8095
Epoch 7/10
167/167 [==============================] - 11s 65ms/step - loss: 0.0742 - accuracy: 0.9698 - val_loss: 0.8537 - val_accuracy:

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/10
167/167 [==============================] - 28s 82ms/step - loss: 0.5095 - accuracy: 0.7666 - val_loss: 0.4223 - val_accuracy: 0.8218
Epoch 2/10
167/167 [==============================] - 11s 65ms/step - loss: 0.3577 - accuracy: 0.8585 - val_loss: 0.4513 - val_accuracy: 0.8262
Epoch 3/10
167/167 [==============================] - 11s 65ms/step - loss: 0.2515 - accuracy: 0.9077 - val_loss: 0.4897 - val_accuracy: 0.8205
Epoch 4/10
167/167 [==============================] - 11s 65ms/step - loss: 0.1576 - accuracy: 0.9461 - val_loss: 0.6771 - val_accuracy: 0.7990
Epoch 5/10
167/167 [==============================] - 11s 65ms/step - loss: 0.1191 - accuracy: 0.9570 - val_loss: 0.9036 - val_accuracy: 0.7837
Epoch 6/10
167/167 [==============================] - 11s 65ms/step - loss: 0.0808 - accuracy: 0.9717 - val_loss: 0.7793 - val_accuracy: 0.7767
Epoch 7/10
167/167 [==============================] - 11s 65ms/step - loss: 0.0683 - accuracy: 0.9747 - val_loss: 0.7910 - val_accuracy:

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Epoch 1/10
167/167 [==============================] - 28s 82ms/step - loss: 0.4969 - accuracy: 0.7735 - val_loss: 0.3807 - val_accuracy: 0.8393
Epoch 2/10
167/167 [==============================] - 11s 66ms/step - loss: 0.3665 - accuracy: 0.8591 - val_loss: 0.3907 - val_accuracy: 0.8301
Epoch 3/10
167/167 [==============================] - 11s 66ms/step - loss: 0.2712 - accuracy: 0.9024 - val_loss: 0.4030 - val_accuracy: 0.8314
Epoch 4/10
167/167 [==============================] - 11s 66ms/step - loss: 0.1905 - accuracy: 0.9375 - val_loss: 0.4588 - val_accuracy: 0.8266
Epoch 5/10
167/167 [==============================] - 11s 66ms/step - loss: 0.1422 - accuracy: 0.9508 - val_loss: 0.5672 - val_accuracy: 0.8275
Epoch 6/10
167/167 [==============================] - 11s 65ms/step - loss: 0.0996 - accuracy: 0.9660 - val_loss: 0.7244 - val_accuracy: 0.8222
Epoch 7/10
167/167 [==============================] - 11s 65ms/step - loss: 0.0828 - accuracy: 0.9698 - val_loss: 0.7076 - val_accuracy:

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/10
167/167 [==============================] - 29s 84ms/step - loss: 0.4979 - accuracy: 0.7692 - val_loss: 0.4243 - val_accuracy: 0.8266
Epoch 2/10
167/167 [==============================] - 11s 66ms/step - loss: 0.3627 - accuracy: 0.8604 - val_loss: 0.4063 - val_accuracy: 0.8266
Epoch 3/10
167/167 [==============================] - 11s 66ms/step - loss: 0.2614 - accuracy: 0.9099 - val_loss: 0.4756 - val_accuracy: 0.8266
Epoch 4/10
167/167 [==============================] - 11s 66ms/step - loss: 0.1787 - accuracy: 0.9384 - val_loss: 0.5276 - val_accuracy: 0.8039
Epoch 5/10
167/167 [==============================] - 11s 66ms/step - loss: 0.1413 - accuracy: 0.9506 - val_loss: 0.6186 - val_accuracy: 0.8227
Epoch 6/10
167/167 [==============================] - 11s 66ms/step - loss: 0.1042 - accuracy: 0.9621 - val_loss: 0.6515 - val_accuracy: 0.8262
Epoch 7/10
167/167 [==============================] - 11s 67ms/step - loss: 0.0810 - accuracy: 0.9715 - val_loss: 0.6905 - val_accuracy:

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/10
167/167 [==============================] - 30s 83ms/step - loss: 0.4853 - accuracy: 0.7857 - val_loss: 0.3749 - val_accuracy: 0.8481
Epoch 2/10
167/167 [==============================] - 11s 66ms/step - loss: 0.3571 - accuracy: 0.8598 - val_loss: 0.3945 - val_accuracy: 0.8463
Epoch 3/10
167/167 [==============================] - 11s 66ms/step - loss: 0.2695 - accuracy: 0.8994 - val_loss: 0.4920 - val_accuracy: 0.8231
Epoch 4/10
167/167 [==============================] - 11s 66ms/step - loss: 0.1802 - accuracy: 0.9383 - val_loss: 0.5082 - val_accuracy: 0.8109
Epoch 5/10
167/167 [==============================] - 11s 66ms/step - loss: 0.1290 - accuracy: 0.9576 - val_loss: 0.6233 - val_accuracy: 0.8174
Epoch 6/10
167/167 [==============================] - 11s 67ms/step - loss: 0.0868 - accuracy: 0.9683 - val_loss: 0.6077 - val_accuracy: 0.8004
Epoch 7/10
167/167 [==============================] - 11s 66ms/step - loss: 0.0799 - accuracy: 0.9683 - val_loss: 0.7532 - val_accuracy:

In [13]:
y_preds = np.array(y_preds)

In [14]:
results = (y_preds.mean(axis=0) > 0.5).astype('int')

In [15]:
output_df = pd.DataFrame({
    'id':test_df['id'],
    'target':results
})

output_df.to_csv(path + "submission3.csv", index=False)